In [1]:
import pandas as pd
import random
import sympy as sym
import numpy as np

pd.set_option('display.max_rows', None)

In [2]:
#Initial data-----------------------------------------------------------------------------------------------------

C10min=0.1     #min C10 value
C10max=10.0   #max C10 value
decimals=2   #number of decimal cases of C10 variable

n=2               #number of C10 variables

st=3.0             #applied stretch
ninc=100        #number of stretch increments
#------------------------------------------------------------------------------------------------------------------

In [3]:
C11 = sym.Symbol('C11')
C12 = sym.Symbol('C12')
C13 = sym.Symbol('C13')
C21 = sym.Symbol('C21')
C22 = sym.Symbol('C22')
C23 = sym.Symbol('C23')
C31 = sym.Symbol('C31')
C32 = sym.Symbol('C32')
C33 = sym.Symbol('C33')

In [4]:
C = sym.Matrix([[C11,C12,C13], [C21,C22,C23], [C31,C32,C33]]) 
I1=sym.trace(C)
Jac=sym.det(C)

#symbolic C10
C10s = sym.Symbol('C10s')

#Generate SEF
SEF=C10s*(I1-3)
#
#Second Piola Kirchoff Stresses
S11=2*sym.diff(SEF,C11)
S12=2*sym.diff(SEF,C12)
S13=2*sym.diff(SEF,C13)
S21=2*sym.diff(SEF,C21)
S22=2*sym.diff(SEF,C22)
S23=2*sym.diff(SEF,C23)
S31=2*sym.diff(SEF,C31)
S32=2*sym.diff(SEF,C32)
S33=2*sym.diff(SEF,C33)
#
S = sym.Matrix([[S11,S12,S13], [S21,S22,S23], [S31,S32,S33]])

In [5]:
#Generate non duplicate random increments and store them in a dataframe
#nincdf = pd.DataFrame(np.round(np.random.uniform(min_inc, max_inc, size=(n, 1)), 0),columns=['ninc'])

In [6]:
#Generate non duplicate random C10 and store them in a dataframe
C10df = pd.DataFrame(np.round(np.random.uniform(C10min, C10max, size=(n, 1)), decimals),columns=['C10'])

#remove duplicates and complete list
C10df=C10df.drop_duplicates()
missing=n-C10df.shape[0]

while missing!=0:
    new = pd.DataFrame(np.round(np.random.uniform(C10min, C10max, size=(missing, 1)), decimals),columns=['C10'])
    C10df=pd.concat([C10df,new], axis=0)
    C10df=C10df.reset_index(drop=True)
    C10df=C10df.drop_duplicates()
    missing=n-C10df.shape[0]

In [10]:
#Calculate stresses
df2 = pd.DataFrame(index=[0], columns=['data', 'C10'])
for i in range(len(C10df)):
    a = np.empty((ninc, 2))
    dst=(st-1)/ninc   #nincdf.ninc[i]
    stj=1.0
    if i==0:
        c=0
    else:
        c=c+ninc  #nincdf.ninc[i-1]
    for j in range(int(ninc)):   #int(nincdf.ninc[i])
        k=c+j
        stj=stj+dst
        #Deformation Gradient
        F=sym.Matrix([[1/(np.sqrt(stj)),0,0], [0,stj,0], [0,0,1/(np.sqrt(stj))]]) 
        Ft=sym.transpose(F)
        C=Ft*F;

        C11v=C[0,0];
        C12v=C[0,1];
        C13v=C[0,2];
        C21v=C[1,0];
        C22v=C[1,1];
        C23v=C[1,2];
        C31v=C[2,0];
        C32v=C[2,1];
        C33v=C[2,2];

        Cauchy=(1/Jac)*(F*S*Ft)
    
        Stress22_Abaqus=Cauchy[1,1]-Cauchy[0,0]
    
        Cauchy_Abaqus=sym.Matrix([[0,0,0], [0, Stress22_Abaqus,0], [0,0,0]]) 
        
        s22_val=Stress22_Abaqus.subs([(C11, C11v), (C12, C12v), 
                                                         (C13, C13v),(C21, C21v), 
                                                         (C22, C22v), (C23, C23v),
                                                         (C31, C31v), (C32, C32v), 
                                                         (C33, C33v),(C10s,C10df.C10[i])])
        #store the data in a numpy array
        a[j,0]=stj
        a[j,1]=s22_val
        
    #within i loop store the numpy array into dataframe
    df2.loc[i, 'data'] = a
    df2.loc[i, 'C10'] = C10df.C10[i]

#print('-----Data----')
#print(df)

In [8]:
#Save data in a .csv file
df2.to_csv('df2.csv', index=False, header=True)

In [ ]:
#old code
#Calculate stresses
df= pd.DataFrame()
for i in range(len(C10df)):
    dst=(st-1)/ninc   #nincdf.ninc[i]
    stj=1.0
    if i==0:
        c=0
    else:
        c=c+ninc  #nincdf.ninc[i-1]
    for j in range(int(ninc)):   #int(nincdf.ninc[i])
        k=c+j
        stj=stj+dst
        #Deformation Gradient
        F=sym.Matrix([[1/(np.sqrt(stj)),0,0], [0,stj,0], [0,0,1/(np.sqrt(stj))]]) 
        Ft=sym.transpose(F)
        C=Ft*F;

        C11v=C[0,0];
        C12v=C[0,1];
        C13v=C[0,2];
        C21v=C[1,0];
        C22v=C[1,1];
        C23v=C[1,2];
        C31v=C[2,0];
        C32v=C[2,1];
        C33v=C[2,2];

        Cauchy=(1/Jac)*(F*S*Ft)
    
        Stress22_Abaqus=Cauchy[1,1]-Cauchy[0,0]
    
        Cauchy_Abaqus=sym.Matrix([[0,0,0], [0, Stress22_Abaqus,0], [0,0,0]]) 
        
        df.at[k,'stretch']=stj
        df.at[k,'stress']=Stress22_Abaqus.subs([(C11, C11v), (C12, C12v), 
                                                         (C13, C13v),(C21, C21v), 
                                                         (C22, C22v), (C23, C23v),
                                                         (C31, C31v), (C32, C32v), 
                                                         (C33, C33v),(C10s,C10df.C10[i])])
        df.at[k,'C10']=C10df.C10[i]

#print('-----Data----')
#print(df)

In [8]:
#Save data in a .csv file
#df.to_csv('df.csv', index=False, header=True)